# Data Preprocessing



In [2]:
import re
# validate the dict data for hero and item
def validate(data):
    id=[]
    for key in data:
        tid=data[key]
        id.append(tid)
    id.sort();
    flag=True
    for i in range(0, len(id)):
        if id[i]!=i:
            print i
            flag=False
    #print id
    return flag
# get the key-id pairs of hero or item data dict to be continuous
def continuous_format(data):
    i=0
    for key in data:
        data[key]=i
        i+=1

# construct item/hero-id based on json data
# reformat the id sequence
def dict_from_json(data, reindex):
    dict_data=dict()
    repeat=0
    for i in range(0, len(data)):
        #name=data[i]['name']
        name=re.sub('(npc_dota_hero_|item_)', '', data[i]['name'])
        #name=re.sub('[npc_|item_]', '', data[i]['name'])
        if name not in dict_data:
            if reindex:
                dict_data[name]=(i-repeat)
            else:
                dict_data[name]=int(data[i]['id'])
        else:
            repeat+=1
            
        
    return dict_data

# for a up-to-date item/hero list, we can http request to
# https://api.opendota.com/api/explorer?sql=SELECT%20*%20FROM%20items
heroes=json.load(open('heroes.json'))['rows']
items=json.load(open('items.json'))['rows']
#print heroes
heroes=dict_from_json(heroes, False)
items=dict_from_json(items, True)

#if validate(heroes):
#    print "heroes continuous id assignment success!"
if validate(items):
    print "items continuous id assignment success!"
#print items
#print heroes
#print items

items continuous id assignment success!


In [3]:
# based on the google drive data
# hero count=113
# item count=1020
import json
import os
import sys

fpath='./data/'

# init some coefficients
# the baseline freq coefficient
a=1.2
b=0.8

# init basic hero-item freq matrix

def maxValInDict(data):
    maxval=(-sys.maxint-1)
    for key in data:
        if data[key]>maxval:
            maxval=data[key]
    return maxval

hcount=maxValInDict(heroes)+1
icount=len(items)
print "hero count: " + str(hcount)
print "item count: " + str(icount)
basic_freq=[]
for i in range(0, hcount):
    basic_freq.append([0]*icount)

# load from data and computer base freq
# the first five heroes in the players list is radiant
for fname in os.listdir(fpath):
    data=json.load(open(fpath+fname))
    players=data['players']
    for i in range(0, len(players)):
        player=players[i]
        heroid=player['hero_id']
        purchase=player['purchase']
        if player['isRadiant']:
            win=player['radiant_win']
        else:
            win=(not player['radiant_win'])
        # be noticed that purchase is a dictionary
        for key in purchase:
            iid=items[key]
            try:
                if win: 
                    basic_freq[heroid][iid]+=a
                else: 
                    basic_freq[heroid][iid]+=b
            except IndexError as e:
                print "file name: " + fname
                print "item id: " + str(iid)
                print "hero id: " + str(heroid)
    


hero count: 121
item count: 292


In [4]:
# predict based on model
# the basic model is a matrix that contains weight 
# returns base h*k item recommendations
# if we do not consider time, rather we consider 
# all the necessary items, then our method should also be effective
# in this case we may recommend more items like 15-20 that captures early-mid-final 
def base_rec(heroes, model, k):
    res=[]
    for h in heroes:
        hifreq=model[h]
        hifreq=hifreq.sort()
        res.append(hifreq[0:k])


[6.8, 0, 0, 0, 0, 21.200000000000006, 27.600000000000005, 2.4000000000000004, 0.8, 0, 0.8, 27.600000000000005, 0, 1.2, 0.8, 4.0, 4.0, 2.4, 26.000000000000007, 0, 15.199999999999998, 0, 0, 27.600000000000005, 0, 0, 0, 26.000000000000007, 0, 6.0, 0, 0, 1.2, 0, 1.2, 0, 0, 0, 3.2, 0, 0.8, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4.8, 0, 0, 0, 3.2, 0, 1.2, 0, 25.200000000000006, 1.6, 0, 0, 0.8, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3.2, 3.2, 0, 0, 15.2, 26.000000000000007, 19.6, 0.8, 3.2, 26.000000000000007, 3.2, 27.600000000000005, 0, 0.8, 0, 0, 0, 0, 0, 0, 0, 15.2, 0, 20.400000000000002, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2.8, 0, 0, 0, 0, 0, 0, 0, 2.0, 4.8, 0, 0, 6.8, 0, 0, 0, 0, 0, 1.2, 0, 0, 0, 0.8, 25.200000000000006, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5.2, 8.0, 0, 26.000000000000007, 3.2, 0, 0, 0.8, 0, 0, 0, 0, 1.2, 0, 0, 0, 13.599999999999998, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.8, 0, 0, 26.000000000000007, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13.59999999999

In [ ]:
# 